# Dynamic Configuration and Chaining in Mirascope

This notebook provides a detailed introduction to using Dynamic Configuration and Chaining in Mirascope. We'll cover various examples ranging from basic usage to more complex chaining techniques.

For more information on these topics, refer to the following documentation:
- [Dynamic Configuration](https://docs.mirascope.io/learn/dynamic_configuration/)
- [Chaining](https://docs.mirascope.io/learn/chaining/)

1. [Setup](#Setup)
2. [Dynamic Configuration](#Dynamic-Configuration)
   - [Basic Usage](#Basic-Usage)
   - [Computed Fields](#Computed-Fields)
   - [Dynamic Tools](#Dynamic-Tools)
3. [Chaining](#Chaining)
   - [Function-based Chaining](#Function-based-Chaining)
   - [Chaining with Computed Fields](#Chaining-with-Computed-Fields)
4. [Advanced Techniques](#Advanced-Techniques)
   - [Combining Dynamic Config and Chaining](#Combining-Dynamic-Config-and-Chaining)
   - [Error Handling in Chains](#Error-Handling-in-Chains)
5. [Conclusion](#Conclusion)

## Setup

First, let's install Mirascope and set up our environment. We'll use OpenAI for our examples, but you can adapt these to other providers supported by Mirascope. For more information on supported providers, see the [Calls documentation](https://docs.mirascope.io/learn/calls/).

In [ ]:
!pip install "mirascope[openai]"

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "your-api-key-here"

## Dynamic Configuration

Dynamic Configuration in Mirascope allows you to modify the behavior of LLM calls at runtime based on input arguments or other conditions. For more details, see the [Dynamic Configuration documentation](https://docs.mirascope.io/learn/dynamic_configuration/).

### Basic Usage

In [1]:
from mirascope.core import openai, prompt_template


@openai.call("gpt-4o-mini")
@prompt_template("Recommend a {genre} book")
def recommend_book(genre: str, creativity: float) -> openai.OpenAIDynamicConfig:
    return {"call_params": {"temperature": creativity}}


# Low creativity recommendation
response = recommend_book("mystery", 0.2)
print("Low creativity:", response.content)

# High creativity recommendation
response = recommend_book("mystery", 0.8)
print("High creativity:", response.content)

Low creativity: I recommend "The Girl with the Dragon Tattoo" by Stieg Larsson. This gripping mystery novel follows journalist Mikael Blomkvist and hacker Lisbeth Salander as they investigate the decades-old disappearance of a wealthy industrialist's niece. The story weaves together themes of family secrets, corruption, and revenge, all while keeping readers on the edge of their seats with its intricate plot and well-developed characters. It's a compelling read that has captivated audiences worldwide. Enjoy!
High creativity: I recommend "The Girl with the Dragon Tattoo" by Stieg Larsson. This gripping mystery novel follows journalist Mikael Blomkvist and hacker Lisbeth Salander as they investigate the decades-old disappearance of a wealthy industrialist's niece. The story weaves together themes of family secrets, corruption, and vengeance, making it a compelling read that keeps you guessing until the end. If you enjoy intricate plots and strong character development, this book is a gre

### Computed Fields

Computed fields allow you to dynamically generate or modify template variables used in your prompt. For more information on prompt templates, see the [Prompts documentation](https://docs.mirascope.io/learn/prompts/).

In [2]:
@openai.call("gpt-4o-mini")
@prompt_template("Recommend a {genre} book with a reading level of {reading_level}")
def recommend_book_by_age(genre: str, age: int) -> openai.OpenAIDynamicConfig:
    reading_level = "adult"
    if age < 12:
        reading_level = "elementary"
    elif age < 18:
        reading_level = "young adult"
    return {"computed_fields": {"reading_level": reading_level}}


response = recommend_book_by_age("fantasy", 15)
print(response.content)

A great recommendation for a young adult fantasy book is **"Six of Crows" by Leigh Bardugo**. This novel features a group of misfits who come together to pull off an impossible heist in a richly imagined world filled with magic, intrigue, and complex characters. The story is fast-paced and full of twists, making it a captivating read for young adults. Plus, the themes of friendship, loyalty, and redemption resonate well with that age group. Enjoy!


### Dynamic Tools

You can dynamically configure which tools are available to the LLM based on runtime conditions. Here's a simple example using a basic tool function:

In [3]:
def format_book(title: str, author: str, genre: str) -> str:
    """Format a book recommendation."""
    return f"{title} by {author} ({genre})"


@openai.call("gpt-4o-mini")
@prompt_template("Recommend a {genre} book")
def recommend_book_with_tool(genre: str) -> openai.OpenAIDynamicConfig:
    return {"tools": [format_book]}


response = recommend_book_with_tool("mystery")
if response.tool:
    print(response.tool.call())
else:
    print(response.content)

The Girl with the Dragon Tattoo by Stieg Larsson (Mystery)


For more advanced usage of tools, including the `BaseToolKit` class, please refer to the [Tools documentation](https://docs.mirascope.io/latest/learn/tools/) and the [Tools and Agents Getting Started Guide](https://docs.mirascope.io/latest/cookbook/agents/).

## Chaining

Chaining in Mirascope allows you to combine multiple LLM calls or operations in a sequence to solve complex tasks. Let's explore two main approaches to chaining: function-based chaining and chaining with computed fields.

### Function-based Chaining

In function-based chaining, you call multiple functions in sequence, passing the output of one function as input to the next. This approach requires you to manage the sequence of calls manually.

In [4]:
@openai.call("gpt-4o-mini")
@prompt_template("Summarize this text: {text}")
def summarize(text: str): ...


@openai.call("gpt-4o-mini")
@prompt_template("Translate this text to {language}: {text}")
def translate(text: str, language: str): ...


original_text = "Long English text here..."
summary = summarize(original_text)
translation = translate(summary.content, "french")
print(translation.content)

Bien sûr ! Veuillez fournir le texte que vous aimeriez résumer, et je serai ravi de vous aider.


### Chaining with Computed Fields

Chaining with computed fields allows you to encapsulate multiple steps within a single function call. This approach provides better traceability and allows you to access the full chain of operations in the response of a single function, rather than having to call and track them separately in sequence.

In [5]:
@openai.call("gpt-4o-mini")
@prompt_template("Summarize this text: {text}")
def summarize(text: str): ...


@openai.call("gpt-4o-mini")
@prompt_template("Translate this text to {language}: {summary}")
def summarize_and_translate(text: str, language: str) -> openai.OpenAIDynamicConfig:
    return {"computed_fields": {"summary": summarize(text)}}


response = summarize_and_translate("Long English text here...", "french")
print("Translation:", response.content)
print(
    "\nComputed fields (including summary):", response.dynamic_config["computed_fields"]
)

Translation: Bien sûr ! Veuillez fournir le texte que vous souhaitez résumer.

Computed fields (including summary): {'summary': OpenAICallResponse(metadata={}, response=ChatCompletion(id='chatcmpl-A6CBgZpwtN1VFqr82MgeOeg01CGue', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Sure! Please provide the text you'd like summarized.", refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1726040180, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_483d39d857', usage=CompletionUsage(completion_tokens=10, prompt_tokens=18, total_tokens=28)), tool_types=None, prompt_template='Summarize this text: {text}', fn_args={'text': 'Long English text here...'}, dynamic_config=None, messages=[{'role': 'user', 'content': 'Summarize this text: Long English text here...'}], call_params={}, call_kwargs={'model': 'gpt-4o-mini', 'messages': [{'role': 'user', 'content': 'Summarize th

As you can see, with computed fields, you get access to both the final translation and the intermediate summary in a single response. This approach provides better traceability and can be particularly useful for debugging and understanding the entire chain of operations without the need to manage multiple separate function calls.

## Advanced Techniques

### Combining Dynamic Config and Chaining

This example demonstrates how to use dynamic configuration within a chain of operations, all encapsulated within a single function call using computed fields.

In [6]:
@openai.call("gpt-4o-mini")
@prompt_template("Generate a {length} {genre} story")
def generate_story(genre: str, length: str): ...


@openai.call("gpt-4o-mini")
@prompt_template("Analyze the tone of this story: {story}")
def analyze_tone(story: str): ...


@openai.call("gpt-4o-mini")
@prompt_template("Rewrite this {genre} story in a {tone} tone: {story}")
def rewrite_story(story: str, genre: str, tone: str) -> openai.OpenAIDynamicConfig:
    return {
        "computed_fields": {"story": story, "genre": genre, "tone": analyze_tone(story)}
    }


original_story = generate_story("mystery", "short").content
rewritten_story = rewrite_story(original_story, "mystery", "suspenseful")

print("Original Story:", original_story)
print("\nRewritten Story:", rewritten_story.content)
print("\nComputed Fields:", rewritten_story.dynamic_config["computed_fields"])

Original Story: **The Vanishing Star**

On the quiet outskirts of the small town of Eldridge Hollow, a mysterious event had caught the attention of the townsfolk. Every twenty years, during the peak of the Perseid meteor shower, the town’s beloved antique clock tower chimed precisely at midnight, signaling the arrival of a rare celestial phenomenon: the Vanishing Star. Legend had it that whoever saw the star would be granted a single wish, but only if they could solve the riddle of the clock tower.

As the night of the meteor shower approached, excitement buzzed through the air. Eleanor, a tenacious young journalist with a penchant for mysteries, decided to investigate. She was intrigued not only by the legend but also by whispers of a recent series of unusual occurrences surrounding the clock tower—a series of stolen artifacts and a sudden disappearance of the town’s beloved historian, Mr. Hargrove.

Eleanor arrived at the clock tower just before midnight on the night of the meteor sh

In this example, we've combined dynamic configuration (using `computed_fields`) with a chain of operations (generate story → analyze tone → rewrite story). The `rewrite_story` function encapsulates this entire chain, allowing us to access all intermediate results in the final response. This approach provides several advantages:

1. Simplicity: The entire chain is managed within a single function call.
2. Traceability: All intermediate steps (original story, analyzed tone) are available in the computed fields, including the full response from `analyze_tone`.
3. Flexibility: We can easily modify or extend the chain by adjusting the computed fields.

This demonstrates the power of combining dynamic configuration with chaining using computed fields, providing a more integrated and manageable approach to complex LLM-based workflows.

### Error Handling in Chains

Implementing robust error handling is crucial in complex chains.

In [7]:
from openai import OpenAIError


@openai.call("gpt-4o-mini")
@prompt_template("Summarize this text: {text}")
def summarize(text: str): ...


@openai.call("gpt-4o-mini")
@prompt_template("Translate this text to {language}: {text}")
def translate(text: str, language: str): ...


def process_text_with_error_handling(text: str, target_language: str):
    try:
        summary = summarize(text).content
    except OpenAIError as e:
        print(f"Error during summarization: {e}")
        summary = text  # Fallback to original text if summarization fails

    try:
        translation = translate(summary, target_language).content
        return translation
    except OpenAIError as e:
        print(f"Error during translation: {e}")
        return summary  # Fallback to summary if translation fails


result = process_text_with_error_handling("Long text here...", "French")
print("Processed Result:", result)

Processed Result: Bien sûr ! Veuillez fournir le texte que vous aimeriez que je résume.


## Conclusion

This notebook has demonstrated various techniques for using Dynamic Configuration and Chaining in Mirascope. These powerful features allow you to create flexible, efficient, and complex LLM-powered applications. By combining these techniques, you can build sophisticated AI systems that can adapt to different inputs and requirements while maintaining robustness and traceability.

Remember to always consider error handling, especially in complex chains, to ensure your applications are resilient to potential issues that may arise during LLM calls or processing steps.